In [4]:
# Load data set and target values
import sys
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/Preprocess')
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/DAlgorithms')

import Preprocessdata
import RFclass

from __future__ import print_function
from __future__ import division

import logging
from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score


from pandas.core.frame import DataFrame
import numpy as np
import pandas as pd


p= Preprocessdata.standardprocess()

# Load data set and target values


p= Preprocessdata.standardprocess()


save_path = '/home/peng/Documents/Project_C/Turbofan/Dataset/'
#names = xrange(0,28)

df = pd.read_csv(save_path + 'train_FD002_2_p20_.txt', header=0)

rad_stat = 2
tree_seed = 'rf'

# drop the first column
df = df.drop(df.columns[[0]], axis=1)


#split the dataset into train and label first

df_array = np.array(df)
target_all = df_array[:,-1]
train_all = df_array[:, 0:-1]

# change the label to be binary Optional
from sklearn.preprocessing import label_binarize

#target_all = label_binarize(target_all, classes = [0,1,2])



#print (train_all[:,0:2])
from sklearn.preprocessing import scale

train_all_scale = scale(train_all)
#print (train_all_scale[:,0:2])

from sklearn.cross_validation import train_test_split
train, test, trainlabel,testlabel = train_test_split( train_all_scale, target_all, test_size=0.3, random_state=42)

#print (df)

In [5]:
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.metrics.classification import accuracy_score
import numpy as np

from dbn import SupervisedDBNClassification




# Splitting data
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Training,
classifier = SupervisedDBNClassification(hidden_layers_structure=[5],
                                         learning_rate_rbm=0.01,
                                         learning_rate=0.001,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=100,
                                         l2_regularization=0.0,
                                         activation_function='relu')
classifier.fit(train, trainlabel)

# Test
y_pred = classifier.predict(test)
print ('Done.\nAccuracy: %f' % accuracy_score(test, y_pred))

[START] Pre-training step:


ValueError: p is nan

In [19]:

y_pred = []
for i in Y_pred:
    y_pred.append(i*2)



y_test = []
for i in Y_test:
    y_test.append(i*2)
    
print ('Done.\nAccuracy: %f' % accuracy_score(y_test, y_pred))    

Done.
Accuracy: 0.356250


In [1]:

#coding:utf-8
import numpy
import random
import math

class DeepBoltzmannMachine:
	def __init__(self,Layer,trainData):
		self.Layer = Layer
		self.trainData = trainData
				
	def pretrain(self,_Alpha=0.01):
		def calcHidList(visList,weight,scale=1.0):
			res = []
			sigmoid = lambda x:1.0/(1.0+math.exp(-x)) 
			for vis in visList:
				res.append(numpy.array([sigmoid(scale*numpy.dot(vis,w)) for w in numpy.transpose(weight)]))
			return res
			
		def calcVisList(hidList,weight,scale=1.0):
			res = []
			sigmoid = lambda x:1.0/(1.0+math.exp(-x)) 
			for hid in hidList:
				res.append(numpy.array([sigmoid(scale*numpy.dot(hid,w)) for w in weight]))
			return res
			
		L = len(self.Layer)
		self.weight = []
		for l in xrange(L-1):
			V = self.Layer[l]
			H = self.Layer[l+1]
			weight = numpy.array([numpy.array([random.uniform(-0.1,0.1) for h in xrange(H)]) for v in xrange(V)])
			print numpy.transpose(weight).shape
			print 'Layer',l,'-',l+1,'train.'
			# 入力データを生成
			visList = self.trainData
			for i in xrange(l):
				visList = calcHidList(visList,self.weight[i],2.0)
			# PersistentContrastiveDivergence用の可視層を生成
			visHatList = numpy.copy(visList)
			# 中間層は2倍する
			visCalcScale = 1.0 if l==0 else 2.0
			hidCalcScale = 1.0 if l==L-2 else 2.0
			
			N = len(visList)
			Alpha = _Alpha
			for iter in xrange(1000):
				# ContrastiveDivergence実行
				visHatList = calcVisList(calcHidList(visHatList,weight,hidCalcScale),weight,visCalcScale)
				#高速化のためのキャッシュ
				cache1 = calcHidList(visList,weight,hidCalcScale)
				cache2 = calcHidList(visHatList,weight,hidCalcScale)
				weightDelta = [numpy.array([0.0 for h in xrange(H)]) for v in xrange(V)]
				for k in xrange(N):
					for v in xrange(V):
						weightDelta[v] += visList[k][v]*cache1[k]
						weightDelta[v] -= visHatList[k][v]*cache2[k]
						#for h in xrange(H):
						#	weightDelta[v][h] += visList[k][v]*cache1[k][h]-visHatList[k][v]*cache2[k][h]
							
						
				wsum = 0
				# 勾配降下
				for v in xrange(V):
					weight[v] += weightDelta[v]/N*Alpha
					wsum += abs(weightDelta[v].sum())
				print wsum,Alpha
				Alpha *= 0.999
			self.weight.append(weight)
	def fixedpointeq(self,vis):
		sigmoid = lambda x:1.0/(1.0+math.exp(-x)) 
		L = len(self.Layer)
		# 固定点方程式をとく
		# 求められるものはp(h|v)の推定値
		# イテレーション回数
		T = 20
		dp = [[numpy.array([random.uniform(0,1) for i in xrange(l)]) for l in self.Layer] for t in xrange(T)]
		# 初期値の設定
		for t in xrange(T):dp[t][0] = numpy.copy(vis)
		for t in xrange(1,T):
			for l in xrange(1,L):
				sum = numpy.copy(self.bias[l])
				for i,w in enumerate(numpy.transpose(self.weight[l-1])):
					sum[i] += numpy.dot(dp[t-1][l-1],w)
				if l+1< L:
					for i,w in enumerate(self.weight[l]):
						sum[i] += numpy.dot(dp[t-1][l+1],w)
				dp[t][l] = numpy.array(map(sigmoid,sum))
		return dp[T-1]
					
	def sampling(self,prev):
		sigmoid = lambda x:1.0/(1.0+math.exp(-x)) 
		N = len(self.trainData)
		L = len(self.Layer)
		# PersistentContrastiveDivergenceでdp表は使いまわす
		# 求められるものはp(v,h)の推定値
		# イテレーション回数
		T = 10
		for t in xrange(T):
			next = [[numpy.array([0.0 for i in xrange(l)]) for l in self.Layer] for n in xrange(N)]
			for n in xrange(N):
				for l in xrange(L):
					sum = numpy.copy(self.bias[l])
					if l-1>=0:
						for i,w in enumerate(numpy.transpose(self.weight[l-1])):
							sum[i] += numpy.dot(prev[n][l-1],w)
					if l+1< L:
						for i,w in enumerate(self.weight[l]):
							sum[i] += numpy.dot(prev[n][l+1],w)
					next[n][l] = numpy.array(map(sigmoid,sum))
			prev,next = next,prev
		return prev
		
	def output(self):
		sigmoid = lambda x:1.0/(1.0+math.exp(-x)) 
		L = len(self.Layer)
		# 求められるものはp(v,h)の推定値
		# イテレーション回数
		T = 20
		dp = [[numpy.array([random.uniform(0,1) for i in xrange(l)]) for l in self.Layer] for t in xrange(T)]
		for t in xrange(1,T):
			for l in xrange(L):
				sum = numpy.copy(self.bias[l])
				if l-1>=0:
					for i,w in enumerate(numpy.transpose(self.weight[l-1])):
						sum[i] += numpy.dot(dp[t-1][l-1],w)
				if l+1< L:
					for i,w in enumerate(self.weight[l]):
						sum[i] += numpy.dot(dp[t-1][l+1],w)
				dp[t][l] = numpy.array(map(sigmoid,sum))
		return dp[T-1]
					
	def finetune(self,Alpha=0.005):
		print 'fine tune.'
		N = len(self.trainData)
		L = len(self.Layer)
		self.bias = [numpy.array([random.uniform(-0.1,0.1) for i in xrange(l)]) for l in self.Layer]
		
		sample = [[numpy.array([random.uniform(0,1) for i in xrange(l)]) for l in self.Layer] for n in xrange(N)]
		for i,vis in enumerate(self.trainData):
			sample[i][0] = numpy.copy(vis)
		for iter in xrange(200):
			weightDelta = [[numpy.array([0.0 for v in xrange(self.Layer[l+1])]) for h in xrange(self.Layer[l])] for l in xrange(L-1)]
			biasDelta = [numpy.array([0.0 for i in xrange(l)]) for l in self.Layer]
			# 固定点方程式
			for vis in self.trainData:
				mu = self.fixedpointeq(vis)	
				for l in xrange(L-1):
					for v in xrange(self.Layer[l]):
						for h in xrange(self.Layer[l+1]):
							weightDelta[l][v][h] += mu[l][v]*mu[l+1][h]/N
				for l in xrange(L):
					biasDelta[l] += mu[l]/N
							
			# サンプリング
			sample = self.sampling(sample)
			for s in sample:
				for l in xrange(L-1):
					for v in xrange(self.Layer[l]):
						for h in xrange(self.Layer[l+1]):
							weightDelta[l][v][h] -= s[l][v]*s[l+1][h]/N
				for l in xrange(L):
					biasDelta[l] -= s[l]/N
			
			wsum = 0
			bsum = 0
			# 更新
			for l in xrange(L-1):
				for v in xrange(self.Layer[l]):
					self.weight[l][v] += Alpha*weightDelta[l][v]
					wsum += abs(weightDelta[l][v].sum())
			for l in xrange(L):
				self.bias[l] += Alpha*biasDelta[l]
				bsum += abs(biasDelta[l].sum())
			
			print wsum,bsum
			Alpha *= 0.999
			
			
if __name__=='__main__':
	V = 5
	N = 20
	input = [numpy.array([1.0*random.randint(0,1) for j in xrange(V)]) for i in xrange(N)]
	
	dbm = DeepBoltzmannMachine([V,10,10],input)
	
	dbm.pretrain()
	
	dbm.finetune()
	
	sum = numpy.array([0.0 for j in xrange(V)])
	for inp in input:
		sum += inp
	print sum/N,dbm.output()[0]

(10, 5)
Layer 0 - 1 train.
61.1423279706 0.01
60.8451099694 0.00999
60.4292881397 0.00998001
60.0183056999 0.00997002999
59.6122221529 0.00996005996001
59.2109364931 0.00995009990005
58.8143514008 0.00994014980015
58.4223722625 0.00993020965035
58.0349070678 0.0099202794407
57.6518663245 0.00991035916126
57.2731629768 0.0099004488021
56.8987123266 0.0098905483533
56.5284319573 0.00988065780494
56.1622416608 0.00987077714714
55.8000633669 0.00986090636999
55.4418210757 0.00985104546362
55.0874407918 0.00984119441816
54.7368504612 0.00983135322374
54.3899799109 0.00982152187051
54.0467607894 0.00981170034864
53.7071265111 0.0098018886483
53.371012201 0.00979208675965
53.0383546421 0.00978229467289
52.7090922251 0.00977251237821
52.3831648987 0.00976273986584
52.0605141228 0.00975297712597
51.7410828226 0.00974322414884
51.4248153444 0.0097334809247
51.1116574132 0.00972374744377
50.8015560914 0.00971402369633
50.4944597392 0.00970430967263
50.1903179762 0.00969460536296
49.8890816442 0.0

In [3]:
import numpy as np
print (np.shape(input))

(20, 5)
